# Categories Customization

In this notebook, we will be creating a custom model for Watson Natural Language Understanding (NLU) categories feature using the Train API.

We will go through following functionalities:
- How to create training data file
- How to train a categories model with NLU train API
- How to get status of the model
- How to use the trained model using NLU Analyze API

To start with you will need an NLU instance and an API key.


## Add your IBM Cloud service credentials here.
- If you use IAM service credentials, leave 'username' set to 'apikey'and set 'password' to the value of your IAM API key.
- If you use pre-IAM service credentials, set the values to your 'username' and 'password'.

Also set 'url' to the URL for your service instance as provided in your service credentials.
See the following instructions for getting your own credentials: https://cloud.ibm.com/docs/watson?topic=watson-iam


In [4]:
username = 'apikey'
password = 'gD5CySLxf5X6hqciNpV0CNzWNezu7pgAqWkbwu8i_g24'
url = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/6ef7f7fe-0789-477b-a5ba-93912ad319ce'

## Create Training Data File

Categories training data requires labels and key phrases. Labels corresponds to the name of the classes. These labels can have hierarchy and are provided as a list of labels, where the order in this list represents the hierarchy. 

Each label can be provided with one or more "key phrases". These key phrases are used to train categories models. The key phrases should be unique per label and should represent the label. The key phrases can be more phrases containing more than 1 word, for example "action movies". 

In [5]:
training_data = [
    {
        "labels": ["Meetings"],
        "key_phrases": ["meetings", "meet", "talk", "discuss", "chat", "zoom", "skype", "webex", "calendar", "time"]
    },
    {
        "labels": ["Meetings", "Initiate"],
        "key_phrases": ["schedule", "book", "book a time", "set up a time", "set something up", "set up", 
                        "booking", "availabile"]
    },
    {
        "labels": ["Meetings", "Amend"],
        "key_phrases": ["amendment", "update", "change of plans", "changes", "change", "updated availability", 
                        "unexpected", "calendar conflict"]
    },
    {
        "labels": ["Meetings", "Amend", "Reschedule"],
        "key_phrases": ["reschedule", "move", "better time", "another time", "move the meeting", "move chat", 
                        "move discussions", "another day"]
    },
    {
        "labels": ["Meetings", "Amend", "Cancel"],
        "key_phrases": ["cancel", "unavailable", "no longer free", "busy", "unable", "unable to attend", "drop"]
    }
]

# Save Training data in a file
import json

training_data_filename = 'training_data.json'

with open(training_data_filename, 'w', encoding='utf-8') as f:
    json.dump(training_data, f)


## Train Categories model with NLU

In [10]:
import json
import ntpath
import requests
import sys
import time

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


######### Create parameters required for making a call to NLU ######### 
feature_to_train = 'categories'

headers = {'Content-Type' : 'multipart/form-data'}

data = {
    'name':'Categories Custom model #1',
    'language':'en',
    'version':'1.0.1'
}

params = {
    'version': '2021-03-08'
}

uri = url + '/v1/models/{}'.format(feature_to_train)


print('\nCreating custom model...')

training_data_filename = 'training_data.json'

######### Make a call to NLU to train the model ######### 
with open(training_data_filename, 'rb') as f:
    response = requests.post(uri,
                         params=params,
                         data=data,
                         files={'training_data': (ntpath.basename(training_data_filename), f, 'application/json')},
                         auth=(username, password),
                         verify=False,
                        )

######### Parse response from NLU ######### 
    
print('Model creation returned: ', response.status_code)

if response.status_code != 201:
    print('Failed to create model')
    print(response.text)
else:
    print('\nCustom model training started...')
    response_json = response.json()
    model_id = response_json['model_id']
    print('Custom Model ID: ', model_id)


Creating custom model...
Model creation returned:  403
Failed to create model
{"code":403,"error":"number of models exceed allowed limit"}


## Retrieve custom categories model by ID

In [14]:
import requests

params = {
    'version': '2021-03-08'
}

uri = url + '/v1/models/categories/' + model_id

######### Make a call to NLU ######### 

response = requests.get(uri, auth=(username, password), params=params, verify=False, headers=headers)

######### Parse response from NLU ######### 

print('Get model returned: ', response.status_code)

response_json = response.json()
print("Response from NLU:\n", json.dumps(response_json, indent=4, sort_keys=True))

Get model returned:  200
Response from NLU:
 {
    "created": "2021-03-10T05:14:53Z",
    "description": null,
    "features": [
        "categories"
    ],
    "language": "en",
    "last_deployed": "2021-03-10T05:18:29Z",
    "last_trained": "2021-03-10T05:14:53Z",
    "model_id": "7fc14083-4c0a-47a0-a589-7fd176c80a3d",
    "model_version": "1.0.1",
    "name": "Categories Custom model #1",
    "status": "available",
    "user_metadata": null,
    "version": "1.0.1",
    "version_description": null,
    "workspace_id": null
}


## Use the model using NLU Analyze API

Once the model is trained, the status from the get request above will turn to `available`. Once the model is `available`, you can make the analyze request using the `model_id`

In [17]:
######### Create request #########

analyze_request_data = {
        "text":"I'd love to discuss this with you over zoom. I'm available tomorrow evening. Let's schedule something!",
        "language": "en",
        "features": {
            "categories": {
                "model": model_id
            }
        }
}

uri = url + '/v1/analyze'

params = {
    'version': '2021-03-08'
}

headers = {'Content-Type' : 'application/json'}

######### Make a call to NLU #########

response = requests.post(uri,
                         params=params,
                         json=analyze_request_data,
                         headers=headers,
                         auth=(username, password),
                         verify=False,
                        )

if response.status_code != 200:
    print('Failed to make request to model. Reason:')
    print(response.text)

else:
    response_json = response.json()

    print("Successfully analyzed request. Response from NLU:\n")
    print(json.dumps(response_json, indent=4, sort_keys=True))

Successfully analyzed request. Response from NLU:

{
    "categories": [
        {
            "label": "/Meetings",
            "score": 0.976741
        },
        {
            "label": "/Meetings/Initiate",
            "score": 0.949461
        }
    ],
    "language": "en",
    "usage": {
        "features": 0,
        "text_characters": 102,
        "text_units": 1
    }
}
